<a href="https://colab.research.google.com/github/davidyaaw/data-analysis/blob/main/abc_xyz_rfm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Проведем RFM анализ не к игрокам а к бандлам из игрового магазина чтобы выявить наиболее релеватные из них для игроков.

Аббревиатура RFM расшифровывается:  

Recency — давность (как давно ваши пользователи что-то у вас покупали);  
Frequency — частота (как часто они у вас покупают);  
Monetary — деньги (общая сумма покупок).  



# Чтение предобработка

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt


df = pd.read_csv("/content/abc_formated.csv",delimiter=';',decimal=',')

df.head(100)



,Unnamed: 0,Наименование товара,Цена,Месяц
0,Игрок 1,Starter pack,2.99,1.0
1,Игрок 2,NaN,NaN,1.0
2,Игрок 3,Hard Currency Pack 1,1.99,1.0
3,Игрок 4,NaN,NaN,1.0
4,Игрок 5,NaN,NaN,1.0
...,...,...,...,...
95,Игрок 16,Starter pack,2.99,5.0
96,Игрок 17,NaN,NaN,5.0
97,Игрок 18,NaN,NaN,5.0
98,Игрок 19,NaN,NaN,5.0


In [2]:
dt = df[['Unnamed: 0','Наименование товара','Цена','Месяц']]

In [3]:
dt.columns = ['player', 'name', 'price', 'month']

В какие то из месяцев конкретные игроки не совершали покупок.

In [4]:
dt.sample(6)

,player,name,price,month
65,Игрок 6,NaN,NaN,4.0
196,Игрок 17,Hard Currency Pack 2,4.99,10.0
188,Игрок 9,NaN,NaN,10.0
157,Игрок 18,NaN,NaN,8.0
201,Игрок 2,NaN,NaN,11.0
19,Игрок 20,NaN,NaN,1.0


In [5]:
dt.name.fillna("no_purchase",inplace=True)
dt.price.fillna(0,inplace=True)

In [6]:
dt['price'] = dt['price'].astype(float)

In [7]:
result = dt.groupby(["name"]).agg({
  # "player": "count",
  "price": "sum",
  "month": "count",
}).reset_index()

In [8]:
result

,name,price,month
0,Hard Currency Pack 1,5.97,3
1,Hard Currency Pack 2,89.82,18
2,Hard Currency Pack 3,159.84,16
3,Hard Currency Pack 4,259.87,13
4,Special bundle 1,143.84,16
5,Starter pack,29.90,10
6,no_purchase,0.00,163


In [9]:
price = dt.groupby(["name"]).agg({
  # "player": "count",
  "price": "min",
}).reset_index()


In [10]:
price

,name,price
0,Hard Currency Pack 1,1.99
1,Hard Currency Pack 2,4.99
2,Hard Currency Pack 3,9.99
3,Hard Currency Pack 4,19.99
4,Special bundle 1,8.99
5,Starter pack,2.99
6,no_purchase,0.00


In [11]:
price = price.drop([6])
price

,name,price
0,Hard Currency Pack 1,1.99
1,Hard Currency Pack 2,4.99
2,Hard Currency Pack 3,9.99
3,Hard Currency Pack 4,19.99
4,Special bundle 1,8.99
5,Starter pack,2.99


In [12]:
columns_titles = ["name","month","price"]
result=result.reindex(columns=columns_titles)

In [13]:
result.columns = ["name","quantity","revenue"]

In [14]:
dt.dtypes


player     object
name       object
price     float64
month     float64
dtype: object

In [15]:
dt

,player,name,price,month
0,Игрок 1,Starter pack,2.99,1.0
1,Игрок 2,no_purchase,0.00,1.0
2,Игрок 3,Hard Currency Pack 1,1.99,1.0
3,Игрок 4,no_purchase,0.00,1.0
4,Игрок 5,no_purchase,0.00,1.0
...,...,...,...,...
235,Игрок 16,Hard Currency Pack 3,9.99,12.0
236,Игрок 17,no_purchase,0.00,12.0
237,Игрок 18,no_purchase,0.00,12.0
238,Игрок 19,Hard Currency Pack 4,19.99,12.0


In [16]:
result = result.drop([6])
result

,name,quantity,revenue
0,Hard Currency Pack 1,3,5.97
1,Hard Currency Pack 2,18,89.82
2,Hard Currency Pack 3,16,159.84
3,Hard Currency Pack 4,13,259.87
4,Special bundle 1,16,143.84
5,Starter pack,10,29.90


## Уберем строки не содержащие данные о покупках т.е. no_purchase.
Month показывается в каком месяце была последняя покупка.

In [17]:
result2 = dt.groupby(["name"]).agg({
  # "player": "count",
  "month": "max",
}).reset_index()
result2

,name,month
0,Hard Currency Pack 1,1.0
1,Hard Currency Pack 2,12.0
2,Hard Currency Pack 3,12.0
3,Hard Currency Pack 4,12.0
4,Special bundle 1,12.0
5,Starter pack,5.0
6,no_purchase,12.0


In [18]:
result2 = result2.drop([6])

In [19]:
result2.columns = ['name', 'last_month']
result2

,name,last_month
0,Hard Currency Pack 1,1.0
1,Hard Currency Pack 2,12.0
2,Hard Currency Pack 3,12.0
3,Hard Currency Pack 4,12.0
4,Special bundle 1,12.0
5,Starter pack,5.0


In [20]:
result

,name,quantity,revenue
0,Hard Currency Pack 1,3,5.97
1,Hard Currency Pack 2,18,89.82
2,Hard Currency Pack 3,16,159.84
3,Hard Currency Pack 4,13,259.87
4,Special bundle 1,16,143.84
5,Starter pack,10,29.90


## Объеденим таблицы чтобы получить выходную с данными о бандле, проданых шт, выручке и последнемом месяце продажи.

In [21]:
df_merge_col = pd.merge(result, result2, on='name')

df_merge_col

,name,quantity,revenue,last_month
0,Hard Currency Pack 1,3,5.97,1.0
1,Hard Currency Pack 2,18,89.82,12.0
2,Hard Currency Pack 3,16,159.84,12.0
3,Hard Currency Pack 4,13,259.87,12.0
4,Special bundle 1,16,143.84,12.0
5,Starter pack,10,29.90,5.0


# RFM Score

recency frequency monetary

r (как давно что-то покупал? Чем более недавно, тем лучше) 1-4  
f (как часто покупал ранее? Чем чаще, тем лучше) 1-4  
m (как много денег потратил на нас? Чем больше, тем лучше) 1-4  

## r -  показывает сколько месяцев прошло от последней покупки

In [22]:
df_merge_col['r']=(12 - df_merge_col['last_month']).abs()
df_merge_col

,name,quantity,revenue,last_month,r
0,Hard Currency Pack 1,3,5.97,1.0,11.0
1,Hard Currency Pack 2,18,89.82,12.0,0.0
2,Hard Currency Pack 3,16,159.84,12.0,0.0
3,Hard Currency Pack 4,13,259.87,12.0,0.0
4,Special bundle 1,16,143.84,12.0,0.0
5,Starter pack,10,29.90,5.0,7.0


## f - показывает частоту покупок , чем больше тем лучше

In [23]:
df_merge_col['f']= df_merge_col['quantity'] / 12
df_merge_col

,name,quantity,revenue,last_month,r,f
0,Hard Currency Pack 1,3,5.97,1.0,11.0,0.250000
1,Hard Currency Pack 2,18,89.82,12.0,0.0,1.500000
2,Hard Currency Pack 3,16,159.84,12.0,0.0,1.333333
3,Hard Currency Pack 4,13,259.87,12.0,0.0,1.083333
4,Special bundle 1,16,143.84,12.0,0.0,1.333333
5,Starter pack,10,29.90,5.0,7.0,0.833333


## m - monetary , сколько денег потратили на бандл. здесь == revenue

In [24]:
df_merge_col['m']= df_merge_col['revenue']
df_merge_col

,name,quantity,revenue,last_month,r,f,m
0,Hard Currency Pack 1,3,5.97,1.0,11.0,0.250000,5.97
1,Hard Currency Pack 2,18,89.82,12.0,0.0,1.500000,89.82
2,Hard Currency Pack 3,16,159.84,12.0,0.0,1.333333,159.84
3,Hard Currency Pack 4,13,259.87,12.0,0.0,1.083333,259.87
4,Special bundle 1,16,143.84,12.0,0.0,1.333333,143.84
5,Starter pack,10,29.90,5.0,7.0,0.833333,29.90


In [25]:
merged_new = pd.merge(df_merge_col, price, on='name')
new_titles = ["name","price","quantity",'revenue','last_month','r','f','m']
merged_new=merged_new.reindex(columns=new_titles)
merged_new

,name,price,quantity,revenue,last_month,r,f,m
0,Hard Currency Pack 1,1.99,3,5.97,1.0,11.0,0.250000,5.97
1,Hard Currency Pack 2,4.99,18,89.82,12.0,0.0,1.500000,89.82
2,Hard Currency Pack 3,9.99,16,159.84,12.0,0.0,1.333333,159.84
3,Hard Currency Pack 4,19.99,13,259.87,12.0,0.0,1.083333,259.87
4,Special bundle 1,8.99,16,143.84,12.0,0.0,1.333333,143.84
5,Starter pack,2.99,10,29.90,5.0,7.0,0.833333,29.90


In [26]:
merged_new['r']= merged_new['r'].astype(int)

 ## Те  бандлы , которые приобрели в последнем месяце или 1 месяц назад получают высокую оценку == 4

In [27]:
def get_r(x):
  if x in [9,10,11,12]:
    return 1
  if x in [5,6,7,8]:
    return 2
  if x in [2,3,4]:
    return 3
  return 4

In [28]:
merged_new['R']=merged_new['r'].apply(get_r)
merged_new

,name,price,quantity,revenue,last_month,r,f,m,R
0,Hard Currency Pack 1,1.99,3,5.97,1.0,11,0.250000,5.97,1
1,Hard Currency Pack 2,4.99,18,89.82,12.0,0,1.500000,89.82,4
2,Hard Currency Pack 3,9.99,16,159.84,12.0,0,1.333333,159.84,4
3,Hard Currency Pack 4,19.99,13,259.87,12.0,0,1.083333,259.87,4
4,Special bundle 1,8.99,16,143.84,12.0,0,1.333333,143.84,4
5,Starter pack,2.99,10,29.90,5.0,7,0.833333,29.90,2


In [29]:
merged_new.groupby('R')['r'].agg(['mean','count'])

,mean,count
R,,
1,11.0,1
2,7.0,1
4,0.0,4


## Функция дискретизации на основе квантилей

recency frequency monetary

r (как давно что-то покупал? Чем более недавно, тем лучше) 1-4  
f (как часто покупал ранее? Чем чаще, тем лучше) 1-4  
m (как много денег потратил на нас? Чем больше, тем лучше) 1-4  

In [30]:
merged_new['F'] = pd.qcut(merged_new['f'],4,labels=False) + 1

In [31]:
merged_new['M'] = pd.qcut(merged_new['m'],4,labels=False) + 1


RFM с плавающей точкой

In [32]:
merged_new['rfm_score'] = (merged_new['R']+merged_new['F']+merged_new['M'])/3

In [33]:
merged_new.sort_values(by='rfm_score',ascending=False)

,name,price,quantity,revenue,last_month,r,f,m,R,F,M,rfm_score
2,Hard Currency Pack 3,9.99,16,159.84,12.0,0,1.333333,159.84,4,3,4,3.666667
1,Hard Currency Pack 2,4.99,18,89.82,12.0,0,1.500000,89.82,4,4,2,3.333333
3,Hard Currency Pack 4,19.99,13,259.87,12.0,0,1.083333,259.87,4,2,4,3.333333
4,Special bundle 1,8.99,16,143.84,12.0,0,1.333333,143.84,4,3,3,3.333333
5,Starter pack,2.99,10,29.90,5.0,7,0.833333,29.90,2,1,1,1.333333
0,Hard Currency Pack 1,1.99,3,5.97,1.0,11,0.250000,5.97,1,1,1,1.000000


### Выбор имен для групп

In [34]:
def rfm_level(df):
    if df['RFM_score'] >= 11:
        return 'Besties <3 '
    elif ((df['RFM_score'] >= 9) and (df['RFM_score'] < 11)):
        return 'Champions'
    elif ((df['RFM_score'] >= 7) and (df['RFM_score'] < 8)):
        return 'Loyal'
    elif ((df['RFM_score'] >= 6) and (df['RFM_score'] < 7)):
        return 'Potential'
    elif ((df['RFM_score'] >= 5) and (df['RFM_score'] < 6)):
        return 'Promising'
    elif ((df['RFM_score'] >= 4) and (df['RFM_score'] < 5)):
        return 'Needs Attention'
    else:
        return 'Require Activation'

Нормализованный RFM < либо == 12

In [35]:
merged_new['RFM_score'] = (merged_new['R']+merged_new['F']+merged_new['M'])
merged_new.sort_values(by='RFM_score',ascending=False)

,name,price,quantity,revenue,last_month,r,f,m,R,F,M,rfm_score,RFM_score
2,Hard Currency Pack 3,9.99,16,159.84,12.0,0,1.333333,159.84,4,3,4,3.666667,11
1,Hard Currency Pack 2,4.99,18,89.82,12.0,0,1.500000,89.82,4,4,2,3.333333,10
3,Hard Currency Pack 4,19.99,13,259.87,12.0,0,1.083333,259.87,4,2,4,3.333333,10
4,Special bundle 1,8.99,16,143.84,12.0,0,1.333333,143.84,4,3,3,3.333333,10
5,Starter pack,2.99,10,29.90,5.0,7,0.833333,29.90,2,1,1,1.333333,4
0,Hard Currency Pack 1,1.99,3,5.97,1.0,11,0.250000,5.97,1,1,1,1.000000,3


In [36]:
merged_new['segment'] = merged_new.apply(rfm_level,axis=1)

## Результат

In [37]:
merged_new.sort_values(by="RFM_score",ascending=False)

,name,price,quantity,revenue,last_month,r,f,m,R,F,M,rfm_score,RFM_score,segment
2,Hard Currency Pack 3,9.99,16,159.84,12.0,0,1.333333,159.84,4,3,4,3.666667,11,Besties <3
1,Hard Currency Pack 2,4.99,18,89.82,12.0,0,1.500000,89.82,4,4,2,3.333333,10,Champions
3,Hard Currency Pack 4,19.99,13,259.87,12.0,0,1.083333,259.87,4,2,4,3.333333,10,Champions
4,Special bundle 1,8.99,16,143.84,12.0,0,1.333333,143.84,4,3,3,3.333333,10,Champions
5,Starter pack,2.99,10,29.90,5.0,7,0.833333,29.90,2,1,1,1.333333,4,Needs Attention
0,Hard Currency Pack 1,1.99,3,5.97,1.0,11,0.250000,5.97,1,1,1,1.000000,3,Require Activation


### Количетво бандлов из под выделенных групп + средняя выручка по группе

In [38]:
merged_new.groupby('segment')['revenue'].agg(['mean','count'])

,mean,count
segment,,
Besties <3,159.84,1
Champions,164.51,3
Needs Attention,29.90,1
Require Activation,5.97,1


In [39]:
#Просмотр групп для F - frequency . Отношение частоты покупок к прибыли .

merged_new.pivot_table(index='F',columns='R',values='revenue',aggfunc='sum')

R,1,2,4
F,,,
1,5.97,29.9,NaN
2,NaN,NaN,259.87
3,NaN,NaN,303.68
4,NaN,NaN,89.82


Напомню. f (как часто покупал ранее? Чем чаще, тем лучше) 1-4

Бандлы которые реже всего покупали.


In [40]:
a = merged_new.loc[merged_new['F'] == 1]
a

,name,price,quantity,revenue,last_month,r,f,m,R,F,M,rfm_score,RFM_score,segment
0,Hard Currency Pack 1,1.99,3,5.97,1.0,11,0.250000,5.97,1,1,1,1.000000,3,Require Activation
5,Starter pack,2.99,10,29.90,5.0,7,0.833333,29.90,2,1,1,1.333333,4,Needs Attention


In [41]:
merged_new

,name,price,quantity,revenue,last_month,r,f,m,R,F,M,rfm_score,RFM_score,segment
0,Hard Currency Pack 1,1.99,3,5.97,1.0,11,0.250000,5.97,1,1,1,1.000000,3,Require Activation
1,Hard Currency Pack 2,4.99,18,89.82,12.0,0,1.500000,89.82,4,4,2,3.333333,10,Champions
2,Hard Currency Pack 3,9.99,16,159.84,12.0,0,1.333333,159.84,4,3,4,3.666667,11,Besties <3
3,Hard Currency Pack 4,19.99,13,259.87,12.0,0,1.083333,259.87,4,2,4,3.333333,10,Champions
4,Special bundle 1,8.99,16,143.84,12.0,0,1.333333,143.84,4,3,3,3.333333,10,Champions
5,Starter pack,2.99,10,29.90,5.0,7,0.833333,29.90,2,1,1,1.333333,4,Needs Attention


### Доля выручки бандла от общей выручки

In [42]:
revenue_sum = merged_new['revenue'].sum()

In [43]:
merged_new['shared'] = merged_new['revenue']/revenue_sum*100
grow = merged_new.sort_values(by='shared', ascending=False )
grow

,name,price,quantity,revenue,last_month,r,f,m,R,F,M,rfm_score,RFM_score,segment,shared
3,Hard Currency Pack 4,19.99,13,259.87,12.0,0,1.083333,259.87,4,2,4,3.333333,10,Champions,37.703848
2,Hard Currency Pack 3,9.99,16,159.84,12.0,0,1.333333,159.84,4,3,4,3.666667,11,Besties <3,23.190761
4,Special bundle 1,8.99,16,143.84,12.0,0,1.333333,143.84,4,3,3,3.333333,10,Champions,20.869363
1,Hard Currency Pack 2,4.99,18,89.82,12.0,0,1.500000,89.82,4,4,2,3.333333,10,Champions,13.031745
5,Starter pack,2.99,10,29.90,5.0,7,0.833333,29.90,2,1,1,1.333333,4,Needs Attention,4.338112
0,Hard Currency Pack 1,1.99,3,5.97,1.0,11,0.250000,5.97,1,1,1,1.000000,3,Require Activation,0.866171


In [44]:
#Check
grow_sum = grow['shared'].sum()
grow_sum

100.0

#ABC-XYZ

In [45]:
t = grow[["name","price","quantity",'revenue','shared']]

t = t.reset_index(drop = True, inplace= False )
t

,name,price,quantity,revenue,shared
0,Hard Currency Pack 4,19.99,13,259.87,37.703848
1,Hard Currency Pack 3,9.99,16,159.84,23.190761
2,Special bundle 1,8.99,16,143.84,20.869363
3,Hard Currency Pack 2,4.99,18,89.82,13.031745
4,Starter pack,2.99,10,29.90,4.338112
5,Hard Currency Pack 1,1.99,3,5.97,0.866171


In [46]:
t

,name,price,quantity,revenue,shared
0,Hard Currency Pack 4,19.99,13,259.87,37.703848
1,Hard Currency Pack 3,9.99,16,159.84,23.190761
2,Special bundle 1,8.99,16,143.84,20.869363
3,Hard Currency Pack 2,4.99,18,89.82,13.031745
4,Starter pack,2.99,10,29.90,4.338112
5,Hard Currency Pack 1,1.99,3,5.97,0.866171


In [47]:
t['shared_grow'] = 0
t

,name,price,quantity,revenue,shared,shared_grow
0,Hard Currency Pack 4,19.99,13,259.87,37.703848,0
1,Hard Currency Pack 3,9.99,16,159.84,23.190761,0
2,Special bundle 1,8.99,16,143.84,20.869363,0
3,Hard Currency Pack 2,4.99,18,89.82,13.031745,0
4,Starter pack,2.99,10,29.90,4.338112,0
5,Hard Currency Pack 1,1.99,3,5.97,0.866171,0


In [48]:
t.loc[t.index[0], 'shared_grow'] = 37.703848
t

,name,price,quantity,revenue,shared,shared_grow
0,Hard Currency Pack 4,19.99,13,259.87,37.703848,37.703848
1,Hard Currency Pack 3,9.99,16,159.84,23.190761,0.000000
2,Special bundle 1,8.99,16,143.84,20.869363,0.000000
3,Hard Currency Pack 2,4.99,18,89.82,13.031745,0.000000
4,Starter pack,2.99,10,29.90,4.338112,0.000000
5,Hard Currency Pack 1,1.99,3,5.97,0.866171,0.000000


In [49]:
tt = t

In [50]:
tt

,name,price,quantity,revenue,shared,shared_grow
0,Hard Currency Pack 4,19.99,13,259.87,37.703848,37.703848
1,Hard Currency Pack 3,9.99,16,159.84,23.190761,0.000000
2,Special bundle 1,8.99,16,143.84,20.869363,0.000000
3,Hard Currency Pack 2,4.99,18,89.82,13.031745,0.000000
4,Starter pack,2.99,10,29.90,4.338112,0.000000
5,Hard Currency Pack 1,1.99,3,5.97,0.866171,0.000000


In [51]:
ttt = t

In [52]:
ttt['shared_grow'] = ttt['shared'].cumsum()
#ttt.at[0, 'shared_grow'] = ttt.at[0, 'shared']


##ABC

ABC анализ группирует объекты по степени важности или неважности продукции для потребителя,

name -- название бандла  
price -- цена единицы  
quantity --  всего приобретено  
revenue -- суммарная выручка   
shared -- доля в процентах от общей выручки  
shared_grow -- доля выручки в процентах с нарастающим итогом   

In [53]:
ttt

,name,price,quantity,revenue,shared,shared_grow
0,Hard Currency Pack 4,19.99,13,259.87,37.703848,37.703848
1,Hard Currency Pack 3,9.99,16,159.84,23.190761,60.894609
2,Special bundle 1,8.99,16,143.84,20.869363,81.763972
3,Hard Currency Pack 2,4.99,18,89.82,13.031745,94.795717
4,Starter pack,2.99,10,29.90,4.338112,99.133829
5,Hard Currency Pack 1,1.99,3,5.97,0.866171,100.000000


~Принцип Парето

А — дают 80% выручки;
В — формируют 15% выручки;
С — дополняют оставшиеся 5%.

In [54]:
def shared_grow_to_category(shared_grow):
    if shared_grow < 80:
        return 'A'
    elif shared_grow < 95:
        return 'B'
    else:
        return 'C'

ttt['Group'] = ttt['shared_grow'].apply(shared_grow_to_category)


In [55]:
ttt

,name,price,quantity,revenue,shared,shared_grow,Group
0,Hard Currency Pack 4,19.99,13,259.87,37.703848,37.703848,A
1,Hard Currency Pack 3,9.99,16,159.84,23.190761,60.894609,A
2,Special bundle 1,8.99,16,143.84,20.869363,81.763972,B
3,Hard Currency Pack 2,4.99,18,89.82,13.031745,94.795717,B
4,Starter pack,2.99,10,29.90,4.338112,99.133829,C
5,Hard Currency Pack 1,1.99,3,5.97,0.866171,100.000000,C


In [56]:
df_sorted = tt.sort_values(by='revenue', ascending=False)
df_sorted['cumulative_revenue'] = df_sorted['revenue'].cumsum()
df_sorted['cumulative_share'] = df_sorted['cumulative_revenue'] / df_sorted['revenue'].sum()
df_sorted

,name,price,quantity,revenue,shared,shared_grow,Group,cumulative_revenue,cumulative_share
0,Hard Currency Pack 4,19.99,13,259.87,37.703848,37.703848,A,259.87,0.377038
1,Hard Currency Pack 3,9.99,16,159.84,23.190761,60.894609,A,419.71,0.608946
2,Special bundle 1,8.99,16,143.84,20.869363,81.763972,B,563.55,0.817640
3,Hard Currency Pack 2,4.99,18,89.82,13.031745,94.795717,B,653.37,0.947957
4,Starter pack,2.99,10,29.90,4.338112,99.133829,C,683.27,0.991338
5,Hard Currency Pack 1,1.99,3,5.97,0.866171,100.000000,C,689.24,1.000000


##XYZ :


### Предобработка исходной таблицы для анализа

In [57]:
#Работаем с исходной таблицой
# группировка данных по name и month и подсчет суммы выручки
revenue_by_month = dt.pivot_table(values='price', index='name', columns='month', aggfunc='sum', fill_value=0)

# группировка данных по name и подсчет общей выручки за год
total_revenue_by_name = dt.groupby('name')['price'].sum()

# вычисление среднемесячной выручки
average_revenue_by_name = total_revenue_by_name / 12

# добавление столбца "Среднее" в таблицу revenue_by_month
revenue_by_month['Среднее'] = revenue_by_month.mean(axis=1)

# объединение таблиц revenue_by_month и average_revenue_by_name по столбцу названию бандла
xyz_pivot = pd.merge(revenue_by_month, average_revenue_by_name, on='name', suffixes=('_по_месяцам', '_за_год'))

print(xyz_pivot)


                        1.0    2.0    3.0    4.0    5.0    6.0    7.0    8.0  \
name                                                                           
Hard Currency Pack 1   5.97   0.00   0.00   0.00   0.00   0.00   0.00   0.00   
Hard Currency Pack 2   0.00   9.98   4.99  14.97   4.99   4.99  14.97   0.00   
Hard Currency Pack 3   0.00   9.99  29.97   9.99  19.98  19.98   9.99  19.98   
Hard Currency Pack 4  19.99   0.00  19.99  19.99  19.99  19.99   0.00  39.98   
Special bundle 1      17.98  26.97   0.00   8.99   8.99   8.99  26.97   0.00   
Starter pack          11.96   8.97   5.98   0.00   2.99   0.00   0.00   0.00   
no_purchase            0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   

                        9.0   10.0   11.0   12.0    Среднее      price  
name                                                                    
Hard Currency Pack 1   0.00   0.00   0.00   0.00   0.497500   0.497500  
Hard Currency Pack 2   9.98   9.98   4.99   9.98   7.485000 

In [58]:
xyz_pivot.drop('price', axis=1, inplace=True)


### Итог.Выручка и усредненная выручка за все месяцы по каждой позиции

In [59]:
xyz_pivot

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,Среднее
name,,,,,,,,,,,,,
Hard Currency Pack 1,5.97,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.497500
Hard Currency Pack 2,0.00,9.98,4.99,14.97,4.99,4.99,14.97,0.00,9.98,9.98,4.99,9.98,7.485000
Hard Currency Pack 3,0.00,9.99,29.97,9.99,19.98,19.98,9.99,19.98,9.99,0.00,9.99,19.98,13.320000
Hard Currency Pack 4,19.99,0.00,19.99,19.99,19.99,19.99,0.00,39.98,39.98,39.98,19.99,19.99,21.655833
Special bundle 1,17.98,26.97,0.00,8.99,8.99,8.99,26.97,0.00,0.00,8.99,17.98,17.98,11.986667
Starter pack,11.96,8.97,5.98,0.00,2.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.491667
no_purchase,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000


In [60]:
xyz_pivot = xyz_pivot.drop(xyz_pivot.index[-1])

In [61]:
xyz_pivot

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,Среднее
name,,,,,,,,,,,,,
Hard Currency Pack 1,5.97,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.497500
Hard Currency Pack 2,0.00,9.98,4.99,14.97,4.99,4.99,14.97,0.00,9.98,9.98,4.99,9.98,7.485000
Hard Currency Pack 3,0.00,9.99,29.97,9.99,19.98,19.98,9.99,19.98,9.99,0.00,9.99,19.98,13.320000
Hard Currency Pack 4,19.99,0.00,19.99,19.99,19.99,19.99,0.00,39.98,39.98,39.98,19.99,19.99,21.655833
Special bundle 1,17.98,26.97,0.00,8.99,8.99,8.99,26.97,0.00,0.00,8.99,17.98,17.98,11.986667
Starter pack,11.96,8.97,5.98,0.00,2.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.491667


In [62]:
xyz_pivot = xyz_pivot.rename(columns={'Среднее': 'mean'})

In [63]:
xyz_pivot['std'] = xyz_pivot.std(axis=1)
xyz_pivot

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,mean,std
name,,,,,,,,,,,,,,
Hard Currency Pack 1,5.97,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.497500,1.650021
Hard Currency Pack 2,0.00,9.98,4.99,14.97,4.99,4.99,14.97,0.00,9.98,9.98,4.99,9.98,7.485000,4.777561
Hard Currency Pack 3,0.00,9.99,29.97,9.99,19.98,19.98,9.99,19.98,9.99,0.00,9.99,19.98,13.320000,8.489867
Hard Currency Pack 4,19.99,0.00,19.99,19.99,19.99,19.99,0.00,39.98,39.98,39.98,19.99,19.99,21.655833,12.795509
Special bundle 1,17.98,26.97,0.00,8.99,8.99,8.99,26.97,0.00,0.00,8.99,17.98,17.98,11.986667,9.236347
Starter pack,11.96,8.97,5.98,0.00,2.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.491667,4.017692


**Чем меньше коэффициент вариации, тем стабильнее спрос на товар.**

In [64]:
xyz_pivot['cv'] = xyz_pivot.apply(lambda row: row['std'] / row['mean'], axis=1)
xyz_pivot

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,mean,std,cv
name,,,,,,,,,,,,,,,
Hard Currency Pack 1,5.97,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.497500,1.650021,3.316625
Hard Currency Pack 2,0.00,9.98,4.99,14.97,4.99,4.99,14.97,0.00,9.98,9.98,4.99,9.98,7.485000,4.777561,0.638285
Hard Currency Pack 3,0.00,9.99,29.97,9.99,19.98,19.98,9.99,19.98,9.99,0.00,9.99,19.98,13.320000,8.489867,0.637377
Hard Currency Pack 4,19.99,0.00,19.99,19.99,19.99,19.99,0.00,39.98,39.98,39.98,19.99,19.99,21.655833,12.795509,0.590857
Special bundle 1,17.98,26.97,0.00,8.99,8.99,8.99,26.97,0.00,0.00,8.99,17.98,17.98,11.986667,9.236347,0.770552
Starter pack,11.96,8.97,5.98,0.00,2.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.491667,4.017692,1.612452


In [78]:
# Нормализация столбца 'cv' методом Min-Max Scaling
min_cv = xyz_pivot['cv'].min()
max_cv = xyz_pivot['cv'].max()
xyz_pivot['cv_normalized'] = (xyz_pivot['cv'] - min_cv) * 100 / (max_cv - min_cv)

In [79]:
xyz_pivot

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,mean,std,cv,Group xyz,cv_normalized
name,,,,,,,,,,,,,,,,,
Hard Currency Pack 1,5.97,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.497500,1.650021,3.316625,Z,100.000000
Hard Currency Pack 2,0.00,9.98,4.99,14.97,4.99,4.99,14.97,0.00,9.98,9.98,4.99,9.98,7.485000,4.777561,0.638285,Y,1.739964
Hard Currency Pack 3,0.00,9.99,29.97,9.99,19.98,19.98,9.99,19.98,9.99,0.00,9.99,19.98,13.320000,8.489867,0.637377,Y,1.706678
Hard Currency Pack 4,19.99,0.00,19.99,19.99,19.99,19.99,0.00,39.98,39.98,39.98,19.99,19.99,21.655833,12.795509,0.590857,X,0.000000
Special bundle 1,17.98,26.97,0.00,8.99,8.99,8.99,26.97,0.00,0.00,8.99,17.98,17.98,11.986667,9.236347,0.770552,Y,6.592433
Starter pack,11.96,8.97,5.98,0.00,2.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.491667,4.017692,1.612452,Z,37.479140


Группа Х — стабильный спрос, объем продаж от месяца к месяцу почти не изменяются  
Группа Y — условно стабильный спрос, т. е. объемы продаж меняются, но без резких скачков  
Группа Z — спрос нестабилен, покупатели приобретают товар от случая к случаю  

Данных совсем немного поэтому коэф. вариации будем оценивать так :

In [81]:
def cv_to_category_normed(cv):
    if cv <= 10:
        return 'X'
    elif cv <= 25:
        return 'Y'
    else:
        return 'Z'

xyz_pivot['Group xyz_normed'] = xyz_pivot['cv_normalized'].apply(cv_to_category_normed)


In [65]:
def cv_to_category(cv):
    if cv < 0.6:
        return 'X'
    elif cv < 1.0:
        return 'Y'
    else:
        return 'Z'

xyz_pivot['Group xyz'] = xyz_pivot['cv'].apply(cv_to_category)


In [82]:
xyz_pivot.sort_values(by='cv',ascending=True)

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,mean,std,cv,Group xyz,cv_normalized,Group xyz_normed
name,,,,,,,,,,,,,,,,,,
Hard Currency Pack 4,19.99,0.00,19.99,19.99,19.99,19.99,0.00,39.98,39.98,39.98,19.99,19.99,21.655833,12.795509,0.590857,X,0.000000,X
Hard Currency Pack 3,0.00,9.99,29.97,9.99,19.98,19.98,9.99,19.98,9.99,0.00,9.99,19.98,13.320000,8.489867,0.637377,Y,1.706678,X
Hard Currency Pack 2,0.00,9.98,4.99,14.97,4.99,4.99,14.97,0.00,9.98,9.98,4.99,9.98,7.485000,4.777561,0.638285,Y,1.739964,X
Special bundle 1,17.98,26.97,0.00,8.99,8.99,8.99,26.97,0.00,0.00,8.99,17.98,17.98,11.986667,9.236347,0.770552,Y,6.592433,X
Starter pack,11.96,8.97,5.98,0.00,2.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.491667,4.017692,1.612452,Z,37.479140,Z
Hard Currency Pack 1,5.97,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.497500,1.650021,3.316625,Z,100.000000,Z


## Матрица ABC-XYZ

### Подготовка таблиц

In [67]:
ttt

,name,price,quantity,revenue,shared,shared_grow,Group
0,Hard Currency Pack 4,19.99,13,259.87,37.703848,37.703848,A
1,Hard Currency Pack 3,9.99,16,159.84,23.190761,60.894609,A
2,Special bundle 1,8.99,16,143.84,20.869363,81.763972,B
3,Hard Currency Pack 2,4.99,18,89.82,13.031745,94.795717,B
4,Starter pack,2.99,10,29.90,4.338112,99.133829,C
5,Hard Currency Pack 1,1.99,3,5.97,0.866171,100.000000,C


In [68]:
abc_xyz = ttt

In [69]:
abc_xyz

,name,price,quantity,revenue,shared,shared_grow,Group
0,Hard Currency Pack 4,19.99,13,259.87,37.703848,37.703848,A
1,Hard Currency Pack 3,9.99,16,159.84,23.190761,60.894609,A
2,Special bundle 1,8.99,16,143.84,20.869363,81.763972,B
3,Hard Currency Pack 2,4.99,18,89.82,13.031745,94.795717,B
4,Starter pack,2.99,10,29.90,4.338112,99.133829,C
5,Hard Currency Pack 1,1.99,3,5.97,0.866171,100.000000,C


In [70]:
merged_table = pd.merge(left=ttt, right=xyz_pivot, on='name', how='left')

merged_table = merged_table[['name', 'Group', 'Group xyz']]


## Final ABC-XYZ Result

In [74]:
merged_table

,name,Group,Group xyz
0,Hard Currency Pack 4,A,X
1,Hard Currency Pack 3,A,Y
2,Special bundle 1,B,Y
3,Hard Currency Pack 2,B,Y
4,Starter pack,C,Z
5,Hard Currency Pack 1,C,Z


import numpy as np
from google.colab import autoviz
df_5952411390947182584 = autoviz.get_df('df_5952411390947182584')

def categorical_histogram(df, colname, figsize=(2, 1.2), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df_5952411390947182584, *['name'], **{})
chart

import numpy as np
from google.colab import autoviz
df_5952411390947182584 = autoviz.get_df('df_5952411390947182584')

def categorical_histogram(df, colname, figsize=(2, 1.2), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df_5952411390947182584, *['Group'], **{})
chart

import numpy as np
from google.colab import autoviz
df_5952411390947182584 = autoviz.get_df('df_5952411390947182584')

def categorical_histogram(df, colname, figsize=(2, 1.2), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df_5952411390947182584, *['Group xyz'], **{})
chart

import numpy as np
from google.colab import autoviz
df_5952411390947182584 = autoviz.get_df('df_5952411390947182584')

def heatmap(df, x_colname, y_colname, figsize=(2, 2), mpl_palette_name='viridis'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  import pandas as pd
  plt.subplots(figsize=figsize)
  df_2dhist = pd.DataFrame({
      x_label: grp[y_colname].value_counts()
      for x_label, grp in df.groupby(x_colname)
  })
  sns.heatmap(df_2dhist, cmap=mpl_palette_name)
  plt.xlabel(x_colname)
  plt.ylabel(y_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = heatmap(df_5952411390947182584, *['name', 'Group'], **{})
chart

import numpy as np
from google.colab import autoviz
df_5952411390947182584 = autoviz.get_df('df_5952411390947182584')

def heatmap(df, x_colname, y_colname, figsize=(2, 2), mpl_palette_name='viridis'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  import pandas as pd
  plt.subplots(figsize=figsize)
  df_2dhist = pd.DataFrame({
      x_label: grp[y_colname].value_counts()
      for x_label, grp in df.groupby(x_colname)
  })
  sns.heatmap(df_2dhist, cmap=mpl_palette_name)
  plt.xlabel(x_colname)
  plt.ylabel(y_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = heatmap(df_5952411390947182584, *['Group', 'Group xyz'], **{})
chart

Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [72]:
table_abc_xyz = merged_table.pivot_table(index=['Group'], columns=['Group xyz'], values=['name'], aggfunc=lambda x: ', '.join(x))
table_abc_xyz.columns = table_abc_xyz.columns.droplevel(level=0)
table_abc_xyz = table_abc_xyz.rename_axis(None, axis=0)
table_abc_xyz = table_abc_xyz.fillna('---')


Матрица ABC-XYZ-анализа. Чем ниже находится товар, тем меньше он приносит выручки.  
 Чем правее находится товар, тем нестабильнее на него спрос.   
Самые непопулярные товары будут в нижнем правом квадрате, а самые популярные — в верхнем левом.  

In [75]:
table_abc_xyz

Group xyz,X,Y,Z
A,Hard Currency Pack 4,Hard Currency Pack 3,---
B,---,"Special bundle 1, Hard Currency Pack 2",---
C,---,---,"Starter pack, Hard Currency Pack 1"


* AX — большая доля прибыли, стабильный спрос.
* AY — большая доля прибыли, колеблющийся спрос.
* AZ — большая доля прибыли, непредсказуемый спрос.
* BX — средние объёмы прибыли, стабильный спрос.
* BY — средние объёмы прибыли, колеблющийся спрос.
* BZ — средние объёмы прибыли, непредсказуемый спрос.
* CX — малозначительная прибыль, стабильный спрос.
* CY — малозначительная прибыль, колеблющийся спрос.
* CZ — малозначительная прибыль, непредсказуемый спрос.

Hard Currency Pack 4	 приносит основной доход.

Start pack нужно вывести хотя бы до уровня BY.

Такая низкая популярность стартер пака негативно влияет на порог вхождения в игру для донатеров , бандл требует активации, переработки. Если поднимать стоимость стартера мы не хотим, то для повышения прибыли  нужно увеличить кол-во продаж. Для этого в стартер можно добавить больше привлекательных предметов. Hard Currency pack 1  совсем не метовый бандл , его нужно изменить тк. за весь год его почти не покупали.

Hard Curerency Pack должен быть хоть как-то видоизмененн , цена должна оставаться по прежнему самой низкой из всех бандлов , требуется увеличить количество продаж.


К тому же не стоит сильно бафать Hard Currency Pack чтобы на фоне аутсайдера по наполнению пользователи склонялись к более дорогой покупок в  виде паков 3 и 4.
